In [1]:
newsletter_topics = ["Gemini vs GPT-4", "LLM Agents", "open-source AI models"]
user_history = [
    "How GPT-4 works", 
    "Phi-2 vs Mistral explained", 
    "Gemma: The Open Source LLM by Google"
]


candidates = [
    {
        "title": "AI Agents, Clearly Explained", 
        "description": "Understanding AI Agents doesn't require a technical background. This video breaks down the evolution from basic LLMs like #ChatGPT to AI Workflows and finally to true #AI Agents through practical, real-world examples. Learn the key differences between these technologies and discover how concepts like RAG and ReAct actually work in simple terms.", 
        "views": 2297904
    },
    {
        "title": "From Zero to Your First AI Agent in 25 Minutes (No Coding)", 
        "description": "Beginner-friendly guide to understanding and building AI agents using n8n. No coding required. Covers tools, APIs, and agent memory.", 
        "views": 1436069
    },
    {
        "title": "How to Build an AI Agent with ChatGPT | Custom GPTs Tutorial", 
        "description": "Hands-on tutorial to build Custom GPTs with practical applications in finance, automation, and prompt engineering.", 
        "views": 6603
    },
         {"title": "Inside Google's Driverless RoboTaxi in San Francisco! AI Future ..", "description": "Inside Google's Driverless RoboTaxi in San Francisco! AI Future ..", "views": 
646975}, 
     {"title": "How Self Driving Cars Work | How Autonomous Vehicles Work | AI | Intellipaat", "description": "Welcome to this video on Self-Driving Cars and how they work! In this video, we will get to know how autonomous vehicles work, shedding light on the cutting-edge technology i.e. AI that powers them. Whether you're a tech enthusiast, a budding AI engineer, or simply curious about the future of transportation, this video is a must-watch.If youre ready to take an interesting journey into the world of self-driving vehicles and autonomous technology, press the play button right now! Don't forget to like, comment, share, and subscribe to our channel for more informative content.",
    "views": 60566
    },
         {"title": "What is Physical AI?", "description": "The power of AI and robotics. Physical AI is, well, AI applied in a physical use -- like the vacuum Roomba. ", "views": 
1924 },
         {"title": "What They Don’t Tell You About Moving to Australia", "description": "Thinking of moving to Australia? Learn the 9 biggest mistakes people make when relocating - plus real tips on visas, cost of living, finding work, and adjusting to life down under.If you’re planning to move to Australia in 2025, watch this first and avoid the surprises no one talks about.", "views": 
114356},
    
    
]


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed user history
user_embeds = model.encode(user_history)
user_profile = user_embeds.mean(axis=0)  # aggregate

# Embed candidates
for vid in candidates:
    vid['embedding'] = model.encode(vid['title'] + " " + vid['description'])

# Embed newsletter topics
topic_embed = model.encode(" ".join(newsletter_topics))


2025-07-23 13:51:09.964397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753278670.240582      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753278670.319694      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
for vid in candidates:
    vid['embedding'] = torch.tensor(model.encode(vid['title'] + " " + vid['description']), dtype=torch.float32)

NameError: name 'torch' is not defined

In [ ]:
# import torch
# import torch.nn as nn

# class FeatureInteractionMLP(nn.Module):
#     def __init__(self, input_dim, hidden_dim=256):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, 1)  # score output
#         )

#     def forward(self, x):
#         return self.net(x)

# model_mlp = FeatureInteractionMLP(input_dim=768*3)


In [ ]:
embed_dim = model.get_sentence_embedding_dimension()

class FeatureInteractionMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)

model_mlp = FeatureInteractionMLP(input_dim=embed_dim * 3)

# Create input tensors
X = []
for vid in candidates:
    x = torch.tensor(
        list(user_profile) + list(topic_embed) + list(vid['embedding']),
        dtype=torch.float32
    )
    X.append(x)

X_tensor = torch.stack(X)
scores = model_mlp(X_tensor).squeeze()


In [ ]:
import pandas as pd

df = pd.DataFrame(candidates)
df['score'] = scores.detach().numpy()
df = df.sort_values(by='score', ascending=False)

df[['title', 'score']].head(5)


In [ ]:
newsletter_topics = ["AI", "Automation", "how Agentic system work"]
user_history = [
    "Autonomus CAR", 
    "How AI robotic works", 
    "Gemma: The Open Source LLM by Google"
]

candidates = [
    {
        "title": "AI Agents, Clearly Explained", 
        "description": "Understanding AI Agents doesn't require a technical background. This video breaks down the evolution from basic LLMs like #ChatGPT to AI Workflows and finally to true #AI Agents through practical, real-world examples. Learn the key differences between these technologies and discover how concepts like RAG and ReAct actually work in simple terms.", 
        "views": 2297904
    },
    {
        "title": "From Zero to Your First AI Agent in 25 Minutes (No Coding)", 
        "description": "Beginner-friendly guide to understanding and building AI agents using n8n. No coding required. Covers tools, APIs, and agent memory.", 
        "views": 1436069
    },
    {
        "title": "How to Build an AI Agent with ChatGPT | Custom GPTs Tutorial", 
        "description": "Hands-on tutorial to build Custom GPTs with practical applications in finance, automation, and prompt engineering.", 
        "views": 6603
    },
         {"title": "Inside Google's Driverless RoboTaxi in San Francisco! AI Future ..", "description": "Inside Google's Driverless RoboTaxi in San Francisco! AI Future ..", "views": 
646975}, 
     {"title": "How Self Driving Cars Work | How Autonomous Vehicles Work | AI | Intellipaat", "description": "Welcome to this video on Self-Driving Cars and how they work! In this video, we will get to know how autonomous vehicles work, shedding light on the cutting-edge technology i.e. AI that powers them. Whether you're a tech enthusiast, a budding AI engineer, or simply curious about the future of transportation, this video is a must-watch.If youre ready to take an interesting journey into the world of self-driving vehicles and autonomous technology, press the play button right now! Don't forget to like, comment, share, and subscribe to our channel for more informative content.",
    "views": 60566
    },
         {"title": "What is Physical AI?", "description": "The power of AI and robotics. Physical AI is, well, AI applied in a physical use -- like the vacuum Roomba. ", "views": 
1924 },
         {"title": "What They Don’t Tell You About Moving to Australia", "description": "Thinking of moving to Australia? Learn the 9 biggest mistakes people make when relocating - plus real tips on visas, cost of living, finding work, and adjusting to life down under.If you’re planning to move to Australia in 2025, watch this first and avoid the surprises no one talks about.", "views": 
114356},
    
    
]


In [ ]:
# Embed user history
user_embeds = model.encode(user_history)
user_profile = user_embeds.mean(axis=0)  # aggregate

# Embed candidates
for vid in candidates:
    vid['embedding'] = model.encode(vid['title'] + " " + vid['description'])

# Embed newsletter topics
topic_embed = model.encode(" ".join(newsletter_topics))

In [ ]:
embed_dim = model.get_sentence_embedding_dimension()

class FeatureInteractionMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)

model_mlp = FeatureInteractionMLP(input_dim=embed_dim * 3)

# Create input tensors
X = []
for vid in candidates:
    x = torch.tensor(
        list(user_profile) + list(topic_embed) + list(vid['embedding']),
        dtype=torch.float32
    )
    X.append(x)

X_tensor = torch.stack(X)
scores = model_mlp(X_tensor).squeeze()


In [ ]:
import pandas as pd

df = pd.DataFrame(candidates)
df['score'] = scores.detach().numpy()
df = df.sort_values(by='score', ascending=False)

df[['title', 'score']].head(5)


# Travel Place Recommendation System Implementation

Based on the video recommendation approach above, let's implement a **transformer-based travel recommendation system** that:

1. **User Query → Embedding** - Convert user travel message to vector
2. **Vector DB Retrieval** - Get top similar places from vector database  
3. **Neural Ranking** - Use MLP to rank retrieved places
4. **Top 30 Results** - Return most suitable places

**Key Differences from Current Implementation:**
- ✅ No manual feature engineering
- ✅ No rule-based fallbacks
- ✅ Pure embedding-based approach
- ✅ Lightweight neural ranker

In [ ]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict, Any

# Mock travel places data (in real implementation, this comes from vector DB)
travel_places = [
    {
        "id": "place_1",
        "name": "Sigiriya Rock Fortress",
        "description": "Ancient rock fortress and palace with stunning frescoes and panoramic views. UNESCO World Heritage site featuring archaeological ruins and historical significance.",
        "category": "cultural",
        "region": "central",
        "rating": 4.8,
        "price_range": "moderate"
    },
    {
        "id": "place_2", 
        "name": "Yala National Park",
        "description": "Premier wildlife sanctuary famous for leopards, elephants, and diverse bird species. Safari experience through varied ecosystems and landscapes.",
        "category": "wildlife",
        "region": "southern",
        "rating": 4.6,
        "price_range": "moderate"
    },
    {
        "id": "place_3",
        "name": "Mirissa Beach",
        "description": "Beautiful golden sand beach perfect for whale watching, surfing, and relaxation. Tropical paradise with crystal clear waters.",
        "category": "beach",
        "region": "southern", 
        "rating": 4.5,
        "price_range": "budget"
    },
    {
        "id": "place_4",
        "name": "Temple of the Tooth",
        "description": "Sacred Buddhist temple housing the relic of Buddha's tooth. Important pilgrimage site with rich cultural and spiritual significance.",
        "category": "spiritual",
        "region": "central",
        "rating": 4.7,
        "price_range": "budget"
    },
    {
        "id": "place_5",
        "name": "Ella Rock Hike",
        "description": "Challenging hiking trail with breathtaking mountain views and tea plantations. Adventure activity for nature enthusiasts.",
        "category": "adventure", 
        "region": "central",
        "rating": 4.9,
        "price_range": "budget"
    },
    {
        "id": "place_6",
        "name": "Galle Fort",
        "description": "Historic Dutch colonial fort with cobblestone streets, boutique shops, and ocean views. Architectural heritage and cultural exploration.",
        "category": "cultural",
        "region": "southern",
        "rating": 4.4,
        "price_range": "moderate"
    },
    {
        "id": "place_7",
        "name": "Nuwara Eliya Tea Plantations", 
        "description": "Scenic hill country with lush tea estates, cool climate, and colonial architecture. Tea tasting and plantation tours.",
        "category": "nature",
        "region": "central",
        "rating": 4.3,
        "price_range": "moderate"
    },
    {
        "id": "place_8",
        "name": "Arugam Bay Surfing",
        "description": "World-class surfing destination with consistent waves and laid-back atmosphere. Beach culture and water sports.",
        "category": "adventure",
        "region": "eastern", 
        "rating": 4.6,
        "price_range": "budget"
    }
]

print(f"Loaded {len(travel_places)} travel places for recommendation")

In [ ]:
# Initialize transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
embed_dim = model.get_sentence_embedding_dimension()

# Create embeddings for all travel places (this would be stored in vector DB)
for place in travel_places:
    place_text = f"{place['name']} {place['description']} {place['category']} {place['region']}"
    place['embedding'] = model.encode(place_text)
    
print(f"Created embeddings with dimension: {embed_dim}")
print("Sample place embedding shape:", travel_places[0]['embedding'].shape)

In [ ]:
class TravelPlaceRanker(nn.Module):
    """
    Lightweight neural ranker for travel places
    Takes: user_query_embedding + user_context_embedding + place_embedding
    Returns: relevance score
    """
    def __init__(self, embedding_dim, hidden_dim=256):
        super().__init__()
        # Simple MLP that takes concatenated embeddings
        self.ranker = nn.Sequential(
            nn.Linear(embedding_dim * 3, hidden_dim),  # user_query + user_context + place
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(), 
            nn.Linear(hidden_dim // 2, 1),  # single relevance score
            nn.Sigmoid()  # score between 0-1
        )
    
    def forward(self, user_query_embed, user_context_embed, place_embed):
        # Concatenate all embeddings
        combined = torch.cat([user_query_embed, user_context_embed, place_embed], dim=-1)
        return self.ranker(combined)

# Initialize the ranker
travel_ranker = TravelPlaceRanker(embedding_dim=embed_dim)
print("Travel Place Ranker initialized")
print(f"Model parameters: {sum(p.numel() for p in travel_ranker.parameters())}")

In [ ]:
def get_travel_recommendations(user_query: str, user_context: Dict[str, Any], top_k: int = 5):
    """
    Main recommendation function following your vision:
    1. User Query → Embedding
    2. Vector DB Similarity Search (simulated here)
    3. Neural Ranking  
    4. Top K Results
    """
    
    # Step 1: Convert user query to embedding
    user_query_embed = model.encode(user_query)
    
    # Step 2: Create user context embedding from preferences/history
    context_text = f"Trip type: {user_context.get('trip_type', '')}, Budget: {user_context.get('budget', '')}, Interests: {', '.join(user_context.get('interests', []))}"
    user_context_embed = model.encode(context_text)
    
    # Step 3: Simulate vector DB similarity search (get top candidates)
    # In real implementation, this would query your vector database
    query_embed_tensor = torch.tensor(user_query_embed, dtype=torch.float32)
    
    # Calculate similarity scores with all places
    similarities = []
    for place in travel_places:
        place_embed_tensor = torch.tensor(place['embedding'], dtype=torch.float32)
        similarity = torch.cosine_similarity(query_embed_tensor, place_embed_tensor, dim=0)
        similarities.append((place, similarity.item()))
    
    # Get top candidates based on similarity (simulate vector DB retrieval)
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_candidates = [place for place, _ in similarities[:10]]  # Top 10 from vector DB
    
    # Step 4: Neural ranking of retrieved candidates
    user_query_tensor = torch.tensor(user_query_embed, dtype=torch.float32).unsqueeze(0)
    user_context_tensor = torch.tensor(user_context_embed, dtype=torch.float32).unsqueeze(0)
    
    ranked_places = []
    for place in top_candidates:
        place_embed_tensor = torch.tensor(place['embedding'], dtype=torch.float32).unsqueeze(0)
        
        # Get neural ranking score
        with torch.no_grad():
            relevance_score = travel_ranker(user_query_tensor, user_context_tensor, place_embed_tensor)
            
        ranked_places.append({
            'place': place,
            'neural_score': relevance_score.item(),
            'similarity_score': similarities[[p[0]['id'] for p, _ in similarities].index(place['id'])][1]
        })
    
    # Sort by neural ranking score
    ranked_places.sort(key=lambda x: x['neural_score'], reverse=True)
    
    return ranked_places[:top_k]

# Test the recommendation system
user_query = "I want to explore cultural heritage sites and ancient temples in Sri Lanka"
user_context = {
    "trip_type": "couple",
    "budget": "moderate", 
    "interests": ["cultural", "historical", "spiritual"]
}

recommendations = get_travel_recommendations(user_query, user_context, top_k=5)

print("🎯 Travel Recommendations:")
print("=" * 50)
for i, rec in enumerate(recommendations, 1):
    place = rec['place']
    print(f"{i}. {place['name']}")
    print(f"   Category: {place['category']} | Region: {place['region']}")
    print(f"   Neural Score: {rec['neural_score']:.3f} | Similarity: {rec['similarity_score']:.3f}")
    print(f"   {place['description'][:100]}...")
    print()

In [ ]:
# Test with different user queries
print("🏖️ Beach and Adventure Query:")
print("=" * 40)

beach_query = "I love beaches, surfing and water sports activities"
beach_context = {
    "trip_type": "solo",
    "budget": "budget",
    "interests": ["beach", "adventure", "water_sports"]
}

beach_recs = get_travel_recommendations(beach_query, beach_context, top_k=3)
for i, rec in enumerate(beach_recs, 1):
    place = rec['place']
    print(f"{i}. {place['name']} - Score: {rec['neural_score']:.3f}")

print("\n🏔️ Nature and Hiking Query:")
print("=" * 40)

nature_query = "Looking for mountain hikes, tea plantations and scenic nature views"
nature_context = {
    "trip_type": "family", 
    "budget": "moderate",
    "interests": ["nature", "adventure", "hiking"]
}

nature_recs = get_travel_recommendations(nature_query, nature_context, top_k=3)
for i, rec in enumerate(nature_recs, 1):
    place = rec['place']
    print(f"{i}. {place['name']} - Score: {rec['neural_score']:.3f}")

## 🚀 Integration with Your Actual System

To implement this in your `pear_ranker.py`, you'll need to:

### **1. Replace Complex Feature Engineering** 
Remove all the manual category mappings, rule-based scoring, and complex encoders from lines 173-320 in `pear_ranker.py`

### **2. Simplified Architecture**
```python
class SimplifiedPEARRanker:
    def __init__(self, embedding_model_name="all-MiniLM-L6-v2"):
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.neural_ranker = TravelPlaceRanker(self.embedding_model.get_sentence_embedding_dimension())
    
    def get_recommendations(self, user_query: str, user_context: dict, top_k: int = 30):
        # 1. Query → Embedding
        # 2. Vector DB Similarity Search  
        # 3. Neural Ranking
        # 4. Return Top K
```

### **3. Vector Database Integration**
- Store all travel place embeddings in your vector DB (Pinecone, Weaviate, ChromaDB)
- Use similarity search to get top 50-100 candidates
- Apply neural ranking to get final top 30

### **4. Key Benefits**
- ✅ **No manual rules** - Pure embedding approach
- ✅ **Lightweight** - Simple MLP ranker  
- ✅ **Scalable** - Works with vector databases
- ✅ **Transformer-based** - Uses semantic understanding
- ✅ **End-to-end learning** - Can be trained on user feedback